In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Your existing imports
import asyncio
import websockets
from aiokafka import AIOKafkaConsumer

KAFKA_BOOTSTRAP_SERVERS = ['165.232.189.85:9092']
KAFKA_TOPIC = 'trade_data'

async def consume_kafka(websocket, path):
    consumer = AIOKafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        auto_offset_reset='earliest',
        enable_auto_commit=True,
    )

    await consumer.start()
    try:
        async for message in consumer:
            await websocket.send(message.value.decode('utf-8'))
    finally:
        await consumer.stop()

async def main():
    async with websockets.serve(consume_kafka, '0.0.0.0', 6789):
        await asyncio.Future()  # Run forever

if __name__ == '__main__':
    asyncio.run(main())
